In [1]:
# Load Libraries
import pandas as pd
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import time
import tensorflow as tf
# tf.compat.v1.disable_eager_execution() # 移除此行以启用 Eager Execution
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error
from tensorflow.python.keras.layers import Lambda
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.layers import Dense, LSTM, TimeDistributed, Flatten, Bidirectional, Input
from sklearn.preprocessing import MinMaxScaler
import h5py
import math
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from google.colab import files
from numpy import savetxt
from tensorflow.keras.utils import plot_model
np.random.seed(20)
tf.random.set_seed(20)

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Function: Create Data History
def create_timesteps(data, n_steps):
	x = []
	y = []
	for i in range(len(data)-1):
		end_ix = i + n_steps
		if end_ix > len(data)-1:
			break
		x1, y1 = data[i:end_ix, :-1], data[end_ix, -1]
		x.append(x1)
		y.append(y1)
	return np.array(x), np.array(y)

In [4]:
# Load the data
data1 = loadmat('/content/drive/My Drive/Colab Notebooks/PINN_Paper_Codes/New_Models/Reference_p5270/Data/p5270_Data.mat')
data2 = loadmat('/content/drive/My Drive/Colab Notebooks/PINN_Paper_Codes/New_Models/Reference_p5270/Data/p5270MechData.mat')
a_t = pd.DataFrame({'Time':data1['LocalAcTime'].ravel().round(2), 'freqQAmpI_filt':data1['freqQAmpI_filt'].ravel(),'physics_out':data1['physics_out'].ravel()})
w_t = pd.DataFrame({'Time':data1['LocalAcTime'].ravel().round(2),'C_filt':data1['C_filt'].ravel()})
m_t = pd.DataFrame({'Time':data1['Time'].ravel().round(2), 'SS':data1['SS'].ravel()})
s_t = pd.DataFrame({'Time':data2['Time'].ravel().round(2), 'V':data2['V'].ravel(), 'V_filt':data2['V_filt'].ravel()})
df = a_t.merge(w_t, on='Time')
df = df.merge(m_t, on='Time')
df = df.merge(s_t, on='Time')
df = df[['freqQAmpI_filt', 'C_filt','V_filt','Time','V','SS','physics_out']]
df_SS = df[['freqQAmpI_filt', 'C_filt', 'SS']]
df_V = df[['freqQAmpI_filt', 'C_filt', 'V_filt']]
df_PhyOut = df[['freqQAmpI_filt', 'C_filt', 'physics_out']]
df_A = df_A = df[['freqQAmpI_filt','C_filt','freqQAmpI_filt']]
df_WV = df[['freqQAmpI_filt','C_filt','C_filt']]
xdf = df[['freqQAmpI_filt', 'C_filt']]
ydf = df[['SS','V_filt']]
print("Input Data:\n", xdf)
print("Target Data:\n",ydf)

Input Data:
         freqQAmpI_filt       C_filt
0         31453.223522  5490.232850
1         31438.277060  5490.239343
2         31421.341616  5490.240838
3         31409.798761  5490.232728
4         31398.815589  5490.222463
...                ...          ...
132394    31055.686766  5457.051859
132395    31041.955345  5457.082380
132396    31035.059531  5457.111463
132397    31020.001291  5457.126443
132398    31012.016780  5457.151945

[132399 rows x 2 columns]
Target Data:
               SS    V_filt
0       5.656166  1.201004
1       5.657157  1.243144
2       5.658317  1.264215
3       5.659499  1.243144
4       5.660404  1.222074
...          ...       ...
132394  5.471059  1.285285
132395  5.474545  0.948161
132396  5.478286  1.158863
132397  5.481751  1.201004
132398  5.484813  0.526756

[132399 rows x 2 columns]


In [5]:
# Preprocessing (SS)
arr = df_SS.to_numpy()
n_steps = 300
xdf, ydf_SS = create_timesteps(arr, n_steps)

# Preprocessing (V)
arr = df_V.to_numpy()
n_steps = 300
xdf, ydf_V = create_timesteps(arr, n_steps)

# Preprocessing (Physics_Out)
arr = df_PhyOut.to_numpy()
n_steps = 300
xdf, phy_outdf = create_timesteps(arr, n_steps)

# Preprocessing (A)
arr = df_A.to_numpy()
n_steps = 300
xdf, Adf = create_timesteps(arr, n_steps)

# Preprocessing (WV)
arr = df_WV.to_numpy()
n_steps = 300
xdf, WVdf = create_timesteps(arr, n_steps)

print('Features shape, X = ', np.shape(xdf))
print('Target shape (SS), Y = ', np.shape(ydf_SS))
print('Target shape (V), Y = ', np.shape(ydf_V))
print('Target shape (A), Y = ', np.shape(Adf))
print('Target shape (WV), Y = ', np.shape(WVdf))
print('Target shape (Physics_Out), Y = ', np.shape(phy_outdf))

# Reshape
in_dim = xdf.shape[1]*xdf.shape[2]
xdf = xdf.reshape((xdf.shape[0], in_dim))
print('After reshaping, X = ', np.shape(xdf))

Features shape, X =  (132099, 300, 2)
Target shape (SS), Y =  (132099,)
Target shape (V), Y =  (132099,)
Target shape (A), Y =  (132099,)
Target shape (WV), Y =  (132099,)
Target shape (Physics_Out), Y =  (132099,)
After reshaping, X =  (132099, 600)


In [6]:
# Split into train-val-test (SS)
x_train, x_test, y_train_SS, y_test_SS = train_test_split(xdf, ydf_SS, test_size=0.2, shuffle=False)
X_train, X_val, Y_train_SS, Y_val_SS = train_test_split(x_train, y_train_SS, test_size=0.125, shuffle=False)

# Split into train-val-test (V)
x_train, x_test, y_train_V, y_test_V = train_test_split(xdf, ydf_V, test_size=0.2, shuffle=False)
X_train, X_val, Y_train_V, Y_val_V = train_test_split(x_train, y_train_V, test_size=0.125, shuffle=False)

# Split into train-val-test (A)
x_train, x_test, y_train_A, y_test_A = train_test_split(xdf, Adf, test_size=0.2, shuffle=False)
X_train, X_val, Y_train_A, Y_val_A = train_test_split(x_train, y_train_A, test_size=0.125, shuffle=False)

# Split into train-val-test (WV)
x_train, x_test, y_train_WV, y_test_WV = train_test_split(xdf, WVdf, test_size=0.2, shuffle=False)
X_train, X_val, Y_train_WV, Y_val_WV = train_test_split(x_train, y_train_WV, test_size=0.125, shuffle=False)

# Split into train-val-test (Physics Out)
x_train, x_test, y_train_Phy, y_test_Phy = train_test_split(xdf, phy_outdf, test_size=0.2, shuffle=False)
X_train, X_val, Y_train_Phy, Y_val_Phy = train_test_split(x_train, y_train_Phy, test_size=0.125, shuffle=False)

# Select Train
X_train = X_train[0:92469]
Y_train_SS = Y_train_SS[0:92469]
Y_train_V = Y_train_V[0:92469]
Y_train_Phy = Y_train_Phy[0:92469]
Y_train_A = Y_train_A[0:92469]
Y_train_WV = Y_train_WV[0:92469]

# Normalize the features
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train))
X_val = pd.DataFrame(scaler.transform(X_val))
x_test = pd.DataFrame(scaler.transform(x_test))

print("Training samples:", np.shape(X_train)[0])
print("Validation samples:", np.shape(X_val)[0])
print("Testing samples:", np.shape(x_test)[0])

Training samples: 92469
Validation samples: 13210
Testing samples: 26420


In [7]:
# Normalize the output: SS
scaler_ss = MinMaxScaler()
Y_train_SS = pd.DataFrame(scaler_ss.fit_transform(Y_train_SS.reshape(-1, 1)))
Y_val_SS = pd.DataFrame(scaler_ss.transform(Y_val_SS.reshape(-1, 1)))
y_test_SS = pd.DataFrame(scaler_ss.transform(y_test_SS.reshape(-1, 1)))

In [8]:
# Normalize the output: V
scaler_v = MinMaxScaler()
Y_train_V = pd.DataFrame(scaler_v.fit_transform(Y_train_V.reshape(-1, 1)))
Y_val_V = pd.DataFrame(scaler_v.transform(Y_val_V.reshape(-1, 1)))
y_test_V = pd.DataFrame(scaler_v.transform(y_test_V.reshape(-1, 1)))

In [9]:
# Normalize WV
scaler_wv = MinMaxScaler()
Y_train_WV = pd.DataFrame(scaler_wv.fit_transform(Y_train_WV.reshape(-1, 1)))
Y_val_WV = pd.DataFrame(scaler_wv.transform(Y_val_WV.reshape(-1, 1)))
y_test_WV = pd.DataFrame(scaler_wv.transform(y_test_WV.reshape(-1, 1)))

In [10]:
# Normalize Amp
scaler_A = MinMaxScaler()
Y_train_A = pd.DataFrame(scaler_A.fit_transform(Y_train_A.reshape(-1, 1)))
Y_val_A = pd.DataFrame(scaler_A.transform(Y_val_A.reshape(-1, 1)))
y_test_A = pd.DataFrame(scaler_A.transform(y_test_A.reshape(-1, 1)))

In [11]:
# Time derivative function
# The original global variable `ss1` caused issues in TensorFlow graph mode (tf.compat.v1.disable_eager_execution).
# This function is now stateless and returns a compatible tensor to avoid graph construction errors.
# If an actual non-zero time derivative is needed, it must be computed in a graph-compatible way,
# for example, by providing the previous state as an input to the layer or using tf.gradients
# with an explicit time variable.
def Time_Derivative(del_t, ss2):
    # Returning a tensor compatible with ss2 to unblock the compilation error.
    # This assumes that for the physics constraint, the derivative term can be considered zero,
    # or serves as a placeholder to fix graph construction.
    return tf.zeros_like(ss2)

**Define Physics-Informed Model**: **Self Learning Constants**

In [12]:
# Define Physics
class gradient_1(tf.keras.layers.Layer):
  def __init__(self):
    super(gradient_1,self).__init__()
    self.SS = tf.keras.layers.Dense(1)
    self.V = tf.keras.layers.Dense(1)
    self.A = tf.keras.layers.Dense(1)
    self.WV = tf.keras.layers.Dense(1)
    self.all_scale = tf.keras.layers.Dense(1)
    # Weights will be added in build method

  def build(self, input_shape):
    super(gradient_1, self).build(input_shape) # Important to call super().build() first
    # Using tf.Variable directly for all c's for consistency and to bypass potential add_weight issues
    self.c1 = tf.Variable(initial_value=tf.ones(shape=(1,), dtype=tf.float32), name="c1", trainable=True)
    self.c2 = tf.Variable(initial_value=tf.ones(shape=(1,), dtype=tf.float32), name="c2", trainable=True)
    self.c3 = tf.Variable(initial_value=tf.ones(shape=(1,), dtype=tf.float32), name="c3", trainable=True)
    self.c4 = tf.Variable(initial_value=tf.ones(shape=(1,), dtype=tf.float32), name="c4", trainable=True)
    initializer = tf.keras.initializers.RandomUniform(10000,30000)
    self.c5 = tf.Variable(initial_value=initializer(shape=(1,), dtype=tf.float32), name="c5", trainable=True)

  def call(self, params):
    ss_pred, v_pred, Amp, wv = params
    ss_pred = self.SS(ss_pred)
    v_pred = self.V(v_pred)
    Amp = self.A(Amp)
    wv = self.WV(wv)

    term1 = Time_Derivative(0.01,ss_pred)
    term2 = tf.divide(self.c2*Amp*self.c3*0.4*tf.constant(math.pi)*wv,self.c5)
    term3 = self.c2*(tf.math.sqrt((1-(tf.divide(Amp,(self.c5)))**2)))
    term4 = tf.divide(tf.constant(math.pi)*Amp*self.c3*0.4*wv,self.c5)
    term5 = term3+term4
    term6 = (self.c4-v_pred)
    term7 = tf.divide(1.,tf.convert_to_tensor(self.c1)) # Explicitly convert to tensor to avoid 'str' error
    term8 = tf.divide(term2,term5)
    term9 = tf.multiply(term8,term6)
    term10 = tf.multiply(term1,term7)
    eqn = term10-term9
    eqn = self.all_scale(eqn)

    return eqn

In [13]:
# Define PINN
def MLP_Physics():

  # Define inputs
  X_train = layers.Input(shape=(600,),name='X_train')
  Amp =layers.Input(shape=(1,),name='Amp')
  wv =layers.Input(shape=(1,),name='wv')

  # Prediction: Shear Stress + Slip Rate
  x = layers.Dense(128,kernel_initializer = 'normal', activation="relu")(X_train)
  x = layers.Dense(64,kernel_initializer = 'normal', activation="relu")(x)
  x = layers.Dense(32,kernel_initializer = 'normal', activation="relu")(x)
  x = layers.Dense(16, kernel_initializer = 'normal', activation="relu")(x)
  x = layers.Dense(8, kernel_initializer = 'normal', activation="relu")(x)
  ss_pred = layers.Dense(1,kernel_initializer = 'normal',activation="linear")(x)
  v_pred = layers.Dense(1,kernel_initializer = 'normal',activation="linear")(x)

  # Model
  grad_out = gradient_1()([ss_pred,v_pred,Amp,wv])
  model = keras.Model(inputs=[X_train,Amp,wv],outputs=[ss_pred,v_pred,grad_out])
  return model

In [14]:
# Model Summary
model=MLP_Physics()
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(1e-3), metrics=['mse','mse','mse'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ X_train             │ (None, 600)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     76,928 │ X_train[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      2,080 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 16)        │        528 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 8)         │        136 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │          9 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 1)         │          9 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Amp (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ wv (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gradient_1          │ (None, 1)         │         10 │ dense_5[0][0],    │
│ (gradient_1)        │                   │            │ dense_6[0][0],    │
│                     │                   │            │ Amp[0][0],        │
│                     │                   │            │ wv[0][0]          │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 87,956 (343.58 KB)

 Trainable params: 87,956 (343.58 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train The Model
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=20,verbose=1, mode='auto')
callbacks_list = [earlystop]
start_time = time.time()
history = model.fit([X_train,Y_train_A,Y_train_WV],[Y_train_SS,Y_train_V,Y_train_Phy], epochs=100, batch_size=32,callbacks=callbacks_list,
                     validation_data=([X_val,Y_val_A,Y_val_WV],[Y_val_SS,Y_val_V,Y_val_Phy]), verbose=1)
end_time = time.time()
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()
print("---Training time: %0.8f seconds ---" % (end_time - start_time))

Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/optimizers/base_optimizer.py:855: UserWarning: Gradients do not exist for variables ['gradient_1/dense_7/kernel', 'gradient_1/dense_7/bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


2890/2890 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - dense_5_loss: 0.0615 - dense_5_mse: 0.0615 - dense_6_loss: 0.0086 - dense_6_mse: 0.0086 - gradient_1_loss: 4.4716e-09 - gradient_1_mse: 4.4716e-09 - loss: 0.0701 - val_dense_5_loss: 0.0059 - val_dense_5_mse: 0.0059 - val_dense_6_loss: 0.0146 - val_dense_6_mse: 0.0145 - val_gradient_1_loss: 1.0696e-12 - val_gradient_1_mse: 1.0696e-12 - val_loss: 0.0205
Epoch 2/100
1339/2890 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - dense_5_loss: 0.0045 - dense_5_mse: 0.0045 - dense_6_loss: 0.0079 - dense_6_mse: 0.0079 - gradient_1_loss: 9.0665e-10 - gradient_1_mse: 9.0665e-10 - loss: 0.0125

In [ ]:
# # Save the model
# print(model.summary())
# model.save('/content/drive/My Drive/Colab Notebooks/PINN_Paper_Codes/Reference_p5270/PINN_2/Exact_PINN_Transfer_model_70_10_20.h5')

In [ ]:
# Model Performance: Training
y_predtrain = model.predict([X_train,Y_train_A,Y_train_WV])
y_predtrain_SS = np.array(y_predtrain)[0,]
y_predtrain_V =np.array(y_predtrain)[1,]

train_r2_SS = r2_score(Y_train_SS, y_predtrain_SS)
train_rmse_SS = np.sqrt(mean_squared_error(Y_train_SS, y_predtrain_SS))

train_r2_V = r2_score(Y_train_V,y_predtrain_V)
train_rmse_V = np.sqrt(mean_squared_error(Y_train_V, y_predtrain_V ))

# Model Performance: Validation
y_predval = model.predict([X_val,Y_val_A,Y_val_WV])
y_predval_SS = np.array(y_predval)[0,]
y_predval_V = np.array(y_predval)[1,]

val_r2_SS = r2_score(Y_val_SS, y_predval_SS)
val_rmse_SS = np.sqrt(mean_squared_error(Y_val_SS, y_predval_SS))

val_r2_V = r2_score(Y_val_V, y_predval_V)
val_rmse_V = np.sqrt(mean_squared_error(Y_val_V, y_predval_V))

# Model Performance: Testing
y_predtest = model.predict([x_test,y_test_A,y_test_WV])
y_predtest_SS =np.array(y_predtest)[0,]
y_predtest_V = np.array(y_predtest)[1,]

test_r2_SS = r2_score(y_test_SS,y_predtest_SS)
test_rmse_SS = np.sqrt(mean_squared_error(y_test_SS, y_predtest_SS))

test_r2_V = r2_score(y_test_V,y_predtest_V)
test_rmse_V = np.sqrt(mean_squared_error(y_test_V, y_predtest_V))

# Print R2 Results
print("R2 scores: Train (SS) - %0.5f, Train (V) - %0.5f" %(train_r2_SS , train_r2_V))
print("R2 scores: Validation (SS) - %0.5f, Validation (V) - %0.5f" %(val_r2_SS , val_r2_V))
print("R2 scores: Testing (SS) - %0.5f, Testing (V) - %0.5f" %(test_r2_SS , test_r2_V))

# Print RSME Results
print("RMSE scores: Train (SS) - %0.5f, Train (V) - %0.5f" %(train_rmse_SS , train_rmse_V))
print("RMSE scores: Validation (SS) - %0.5f, Validation (V) - %0.5f" %(val_rmse_SS , val_rmse_V))
print("RMSE scores: Testing (SS) - %0.5f, Testing (V) - %0.5f" %(test_rmse_SS , test_rmse_V))

In [ ]:
# Inverse scaling: SS
Y_train_SS = scaler_ss.inverse_transform(Y_train_SS)
Y_val_SS = scaler_ss.inverse_transform(Y_val_SS)
y_test_SS = scaler_ss.inverse_transform(y_test_SS)
y_predtrain_SS = scaler_ss.inverse_transform(y_predtrain_SS)
y_predval_SS = scaler_ss.inverse_transform(y_predval_SS)
y_predtest_SS = scaler_ss.inverse_transform(y_predtest_SS)

In [ ]:
# Inverse scaling: V
Y_train_V = scaler_v.inverse_transform(Y_train_V)
Y_val_V = scaler_v.inverse_transform(Y_val_V)
y_test_V = scaler_v.inverse_transform(y_test_V)
y_predtrain_V = scaler_v.inverse_transform(y_predtrain_V)
y_predval_V = scaler_v.inverse_transform(y_predval_V)
y_predtest_V = scaler_v.inverse_transform(y_predtest_V)

In [ ]:
# Overall plot: SS & V
ttime = df['Time'][n_steps:]
t1, testtime = train_test_split(ttime, test_size=0.2, shuffle=False)
traintime, valtime = train_test_split(t1, test_size=0.125, shuffle=False)

traintime = traintime[0:92469]

fig = plt.figure(1, figsize=(20,6))
plt.plot(ttime, ydf_SS)
plt.plot(traintime, y_predtrain_SS)
plt.plot(valtime, y_predval_SS)
plt.plot(testtime, y_predtest_SS)
plt.xlabel('Time (s)')
plt.ylabel('Shear Stress (MPa)')
plt.legend(['Ground truth', 'Train Prediction', 'Validation Prediction', 'Test Prediction'])

fig = plt.figure(2, figsize=(20,6))
plt.plot(ttime, ydf_V)
plt.plot(traintime, y_predtrain_V)
plt.plot(valtime,  y_predval_V)
plt.plot(testtime, y_predtest_V)
plt.xlabel('Time (s)')
plt.ylabel('Slip Rate (um/s)')
plt.legend(['Ground truth', 'Train Prediction', 'Validation Prediction', 'Test Prediction'])

In [ ]:
# Individual plot: SS

def results_plot(fig_no, t, gt, pred, title, col):
  fig = plt.figure(fig_no, figsize=(20,6))
  plt.plot(t, gt)
  plt.plot(t, pred, col)
  plt.xlabel('Time (s)')
  plt.ylabel('Shear Stress (MPa)')
  plt.legend(['Ground truth', 'Predicted'])
  plt.title(title)

results_plot(1, traintime, Y_train_SS, y_predtrain_SS, 'Training data', '')
results_plot(2, valtime, Y_val_SS, y_predval_SS, 'Validation data', 'g')
results_plot(3, testtime, y_test_SS, y_predtest_SS, 'Testing data', 'r')

In [ ]:
# Individual plot: V

def results_plot(fig_no, t, gt, pred, title, col):
  fig = plt.figure(fig_no, figsize=(20,6))
  plt.plot(t, gt)
  plt.plot(t, pred, col)
  plt.xlabel('Time (s)')
  plt.ylabel('Slip Rate (um/s)')
  plt.legend(['Ground truth', 'Predicted'])
  plt.title(title)

results_plot(1, traintime, Y_train_V, y_predtrain_V, 'Training data', '')
results_plot(2, valtime, Y_val_V, y_predval_V, 'Validation data', 'g')
results_plot(3, testtime, y_test_V, y_predtest_V, 'Testing data', 'r')

In [ ]:
# Extract Weights
names = [weight.name for layer in model.layers for weight in layer.weights]
weights = model.get_weights()

for name, weight in zip(names, weights):
    print(name, weight.shape)

In [ ]:
# Display Weights
model.get_weights()

In [ ]:
# ML Learned Constants

# Normal Stress
r1 = 10/5.89
cols = ['NS']
data = pd.DataFrame(np.array([[-1.63]]),columns=cols)
NS = scaler_ss.inverse_transform(data)
NS = NS*r1

# Shear Loading Velocity
dt = 0.01
cols = ['VL']
data = pd.DataFrame(np.array([[1.07]]),columns=cols)
VL = scaler_v.inverse_transform(data)
VL = (VL*dt)*(2*r1)

# Loading Stiffness
K = ((2.1)/10)/(3*r1)

# Density (kg/cm3)
rho = (1.42*10*1000)/(3*r1)

print("Normal Stress:",NS)
print("Shear Loading Velocity:",VL)
print("Loading Stiffness:",K)
print("Density:",rho)
print("A_Intact:",12745)

In [ ]:
# # Extract Data

# # Time
# savetxt('Whole_time.csv', ttime, delimiter=',')
# savetxt('traintime.csv', traintime, delimiter=',')
# savetxt('valtime.csv', valtime, delimiter=',')
# savetxt('testtime.csv', testtime, delimiter=',')

# # SS
# savetxt('Y_train_SS.csv', Y_train_SS, delimiter=',')
# savetxt('Y_val_SS.csv', Y_val_SS, delimiter=',')
# savetxt('y_test_SS.csv', y_test_SS, delimiter=',')
# savetxt('y_predtrain_SS.csv', y_predtrain_SS, delimiter=',')
# savetxt('y_predval_SS.csv', y_predval_SS, delimiter=',')
# savetxt('y_predtest_SS.csv', y_predtest_SS, delimiter=',')

# # V
# savetxt('Y_train_V.csv', Y_train_V, delimiter=',')
# savetxt('Y_val_V.csv', Y_val_V, delimiter=',')
# savetxt('y_test_V.csv', y_test_V, delimiter=',')
# savetxt('y_predtrain_V.csv', y_predtrain_V, delimiter=',')
# savetxt('y_predval_V.csv', y_predval_V, delimiter=',')
# savetxt('y_predtest_V.csv', y_predtest_V, delimiter=',')

# # Download
# files.download('Whole_time.csv')
# files.download('traintime.csv')
# files.download('valtime.csv')
# files.download('testtime.csv')

# files.download('Y_train_SS.csv')
# files.download('Y_val_SS.csv')
# files.download('y_test_SS.csv')
# files.download('y_predtrain_SS.csv')
# files.download('y_predval_SS.csv')
# files.download('y_predtest_SS.csv')

# files.download('Y_train_V.csv')
# files.download('Y_val_V.csv')
# files.download('y_test_V.csv')
# files.download('y_predtrain_V.csv')
# files.download('y_predval_V.csv')
# files.download('y_predtest_V.csv')